In [24]:
import pandas as pd
from scipy import stats

df=pd.read_csv('alldata.csv')
df

,Unnamed: 0,Drug Name,Drug ID,Cell Line Name,Cosmic ID,TCGA Classification,Tissue,Tissue Sub-type,IC50,AUC,Max Conc,RMSE,Z score,Dataset Version
0,0,Camptothecin,1003,TE-5,735784,ESCA,aero_digestive_tract,oesophagus,-2.555311,0.834076,0.1,0.087242,-0.161952,NaN
1,1,Camptothecin,1003,EC-GI-10,753555,ESCA,aero_digestive_tract,oesophagus,-3.125664,0.804942,0.1,0.082368,-0.472096,NaN
2,2,Camptothecin,1003,HCE-4,753559,ESCA,aero_digestive_tract,oesophagus,-3.536140,0.778670,0.1,0.087080,-0.695303,NaN
3,3,Camptothecin,1003,KYSE-140,753573,ESCA,aero_digestive_tract,oesophagus,-0.402974,0.960632,0.1,0.085004,1.008434,NaN
4,4,Camptothecin,1003,KYSE-410,753574,ESCA,aero_digestive_tract,oesophagus,0.605862,0.944780,0.1,0.053907,1.557014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90996,6091,N-acetyl cysteine,2499,FU97,1290806,STAD,digestive_system,stomach,10.541691,0.960992,2000.0,0.062524,0.626452,GDSC2
90997,6092,N-acetyl cysteine,2499,NUGC-4,1298357,STAD,digestive_system,stomach,9.745149,0.916434,2000.0,0.094419,-0.286110,GDSC2
90998,6093,N-acetyl cysteine,2499,OCUM-1,1298358,STAD,digestive_system,stomach,10.959875,0.967442,2000.0,0.104618,1.105546,GDSC2
90999,6094,N-acetyl cysteine,2499,HSC-39,1322224,STAD,digestive_system,stomach,8.342217,0.966459,2000.0,0.129949,-1.893383,GDSC2


In [15]:
from sklearn.preprocessing import OneHotEncoder
#get the categorical data
cat = df.select_dtypes(include='O').keys()
df=df[['Drug Name', 'Cell Line Name', 'Tissue','Tissue Sub-type']]
drugnamecount=df['Cell Line Name'].value_counts()
print(drugnamecount.size)
#encode with get_dummies function
ohe=pd.get_dummies(data=df,columns=['Drug Name', 'Cell Line Name', 'Tissue','Tissue Sub-type'])
print(ohe)


904
       Drug Name_123138  Drug Name_123829  Drug Name_150412  \
0                     0                 0                 0   
1                     0                 0                 0   
2                     0                 0                 0   
3                     0                 0                 0   
4                     0                 0                 0   
...                 ...               ...               ...   
90996                 0                 0                 0   
90997                 0                 0                 0   
90998                 0                 0                 0   
90999                 0                 0                 0   
91000                 0                 0                 0   

       Drug Name_5-Fluorouracil  Drug Name_5-azacytidine  Drug Name_50869  \
0                             0                        0                0   
1                             0                        0                0   
2       

In [55]:

Y=df[['Drug ID']].astype('category')
Y

,Drug ID
0,1003
1,1003
2,1003
3,1003
4,1003
...,...
90996,2499
90997,2499
90998,2499
90999,2499


In [56]:
X=ohe.iloc[:,289:]
print(X.head())

   Cell Line Name_22RV1  Cell Line Name_23132-87  Cell Line Name_42-MG-BA  \
0                     0                        0                        0   
1                     0                        0                        0   
2                     0                        0                        0   
3                     0                        0                        0   
4                     0                        0                        0   

   Cell Line Name_451Lu  Cell Line Name_5637  Cell Line Name_639-V  \
0                     0                    0                     0   
1                     0                    0                     0   
2                     0                    0                     0   
3                     0                    0                     0   
4                     0                    0                     0   

   Cell Line Name_647-V  Cell Line Name_697  Cell Line Name_769-P  \
0                     0                   0    

In [57]:
x2=X.iloc[:,916:918]
x2

,Tissue_urogenital_system,Tissue Sub-type_B_cell_leukemia
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
90996,0,0
90997,0,0
90998,0,0
90999,0,0


In [58]:
x1=X.drop(['Cell Line Name_22RV1','Tissue_aero_digestive_tract','Tissue Sub-type_B_cell_leukemia'],axis='columns')
print(x1.head())

   Cell Line Name_23132-87  Cell Line Name_42-MG-BA  Cell Line Name_451Lu  \
0                        0                        0                     0   
1                        0                        0                     0   
2                        0                        0                     0   
3                        0                        0                     0   
4                        0                        0                     0   

   Cell Line Name_5637  Cell Line Name_639-V  Cell Line Name_647-V  \
0                    0                     0                     0   
1                    0                     0                     0   
2                    0                     0                     0   
3                    0                     0                     0   
4                    0                     0                     0   

   Cell Line Name_697  Cell Line Name_769-P  Cell Line Name_786-0  \
0                   0                     0    

In [59]:
#Splitting dataset into training and testing dataset
import tensorflow as tf
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [66]:
#Initialising ANN
ann = tf.keras.models.Sequential()
from keras.layers import Dense
ann.add(Dense(units=6,activation="relu")) #Adding First Hidden Layer
ann.add(Dense(units=6,activation="relu")) #Adding Second Hidden Layer
ann.add(Dense(units=1,activation="sigmoid"))#Adding output layer
ann.compile(optimizer="rmsprop",loss="mean_squared_error",metrics=['accuracy']) #Compiling the ANN

In [67]:
#Fit the ANN
ann.fit(X_train,Y_train,batch_size=32,epochs = 10) #Fitting the ANN

Epoch 1/10
2275/2275 [==============================] - 8s 3ms/step - loss: 2608814.5000 - accuracy: 0.0000e+00
Epoch 2/10
2275/2275 [==============================] - 8s 3ms/step - loss: 2608766.5000 - accuracy: 0.0000e+00
Epoch 3/10
2275/2275 [==============================] - 10s 4ms/step - loss: 2608768.0000 - accuracy: 0.0000e+00
Epoch 4/10
2275/2275 [==============================] - 8s 3ms/step - loss: 2608768.5000 - accuracy: 0.0000e+00
Epoch 5/10
2275/2275 [==============================] - 11s 5ms/step - loss: 2608770.0000 - accuracy: 0.0000e+00
Epoch 6/10
2275/2275 [==============================] - 9s 4ms/step - loss: 2608767.0000 - accuracy: 0.0000e+00
Epoch 7/10
2275/2275 [==============================] - 9s 4ms/step - loss: 2608766.7500 - accuracy: 0.0000e+00
Epoch 8/10
2275/2275 [==============================] - 8s 3ms/step - loss: 2608768.5000 - accuracy: 0.0000e+00
Epoch 9/10
2275/2275 [==============================] - 6s 3ms/step - loss: 2608768.0000 - accuracy: 0

In [68]:
pred=ann.predict(X_test)
print(pd.DataFrame(pred))

569/569 [==============================] - 1s 2ms/step
         0
0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
...    ...
18196  1.0
18197  1.0
18198  1.0
18199  1.0
18200  1.0

[18201 rows x 1 columns]


In [69]:
print(ann.evaluate(X_test,Y_test))

569/569 [==============================] - 1s 2ms/step - loss: 2593896.7500 - accuracy: 0.0000e+00
[2593896.75, 0.0]
